In [ ]:
import snntorch

device = "cpu"
class Model(nn.Module):
    def __init__(self, beta: float = 0.95):
        super().__init__()
        self.fc = nn.Linear(n_neurons, n_neurons)
        self.lif = snntorch.Leaky(beta=beta)
        self.mem = self.lif.init_leaky()

    def forward(self, x):
        cur = self.fc(x)
        spk, self.mem = self.lif(cur, self.mem)
        return spk


torch.autograd.set_detect_anomaly(False)

model = Model()
model = torch.compile(model)#, mode="max-autotune")
model = model.to(device)
input_static = torch.randn(n_steps, batch_size, n_neurons).to(device) # n_steps

output = []
for step in range(n_steps):
    output.append(model(input_static[step]))
    
torch.stack(output).sum().backward()

        # for inp in x:
        #                 output.append(spk)
        # return torch.stack(output)


In [ ]:
from norse.torch.module.lif import LIF
from norse.torch import SequentialState
import norse

model = SequentialState(
    nn.Linear(n_neurons, n_neurons),
    LIF(),
)
model = model.to(device)
model = torch.compile(model)#, mode="max-autotune")
input_static = torch.randn(n_steps, batch_size, n_neurons).to(device)

model(input_static)

In [ ]:
import torch 

import sinabs.layers as sl


model = nn.Sequential(
    nn.Linear(10, 10),
    sl.LIF(tau_mem=torch.tensor(10.)),
)